<a href="https://colab.research.google.com/github/saikumar-chev/Advanced-Notepad/blob/main/LATS_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lifespan Age Transformation Synthesis Demo

This Colab notebook demonstrates the capabilities of the GAN architecture proposed in our paper.

This colab lets you try our method on your own image!

First, let's download the github repository and install all dependencies.

In [ ]:
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/
!pip3 install -r requirements.txt

Now let's download the pretrained models for males and females.

In [ ]:
!python download_models.py

Here, we import libraries and set options.

In [ ]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

Don't worry about this message above,
```
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-c9d47a98-bdba-4a5f-9f0a-e1437c7228b6.json
```
everything is perfectly fine...

Next on, we call the data loader and the visualizer class that generates the video from the network outputs.

In [ ]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

Here, we define our model.

NOTE: if you plan to try the method for a female, change opt.name to 'females_model'.

In [ ]:
opt.name = 'males_model' # change to 'females_model' if you're trying the code on a female image
model = create_model(opt)
model.eval()

OK, it's time to upload your image.

For best results, use images according to the following guidelines:

1. The image should contain a single face.
2. Image was taken from a digital camera (phone cameras are fine). Old images from film cameras would produce low quality results.
3. Pure RGB images only. No black & white, grayscale, sepia, or filtered images (e.g. Instagram filters).
4. Person's head should directly face the camera. Looking sideways/downwards/upwards degrades the results.
5. The person's face should not be occluded (or partially occluded) by any item.
6. Both eyes should be open and visible. (eyeglasses are ok, no sunglasses)

Your uploaded images are local to the Colab instance and are not accessible by the paper authors

In [ ]:
# upload your image (the code supports only a single image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

Finally, we preprocess the image, run the network, and save the result.

In [ ]:
!pip install "Pillow<10.0"

In [ ]:
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

Let's display at the results.

NOTE: if you're using chrome, uncomment the lines below. For some reason, mp4 files won't display on chrome browser, so we need to convert to webm.

In [ ]:
use_webm = False
# For some unknown reason the mp4 video is not displayed on chrome
# If you have chrome, uncomment the following lines to convert the
# result to webm for display purposes

# !pip3 install webm
# webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
# !webm -i $out_path $webm_out_path
# use_webm = True

from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width={0} controls>
      <source src="{1}" type="{2}">
</video>
""".format(opt.fineSize, data_url, video_type))

You can download the result if you want

In [ ]:
files.download(out_path)

In [ ]:
!pip install os

In [ ]:
file_path = "util/preprocess_itw_im.py"
if not os.path.exists(file_path):
    print(f"Error: File {file_path} not found!")
else:
    with open(file_path, 'r') as file:
        content = file.read()
    if "Image.ANTIALIAS" in content:
        content = content.replace("Image.ANTIALIAS", "Image.Resampling.LANCZOS")
        with open(file_path, 'w') as file:
            file.write(content)
        print("Patched Image.ANTIALIAS to Image.Resampling.LANCZOS")
    else:
        print("No Image.ANTIALIAS found in the file")

In [ ]:
import os
import importlib
from PIL import Image
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from google.colab import files
from IPython.display import HTML
from base64 import b64encode

# Step 1: Clone the GitHub repository
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/

# Step 2: Install required packages
!pip install -r requirements.txt

# Step 3: Patch deprecated Image.ANTIALIAS to Image.Resampling.LANCZOS
file_path = "util/preprocess_itw_im.py"
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    if "Image.ANTIALIAS" in content:
        content = content.replace("Image.ANTIALIAS", "Image.Resampling.LANCZOS")
        with open(file_path, 'w') as file:
            file.write(content)
        print("Successfully patched util/preprocess_itw_im.py")
    else:
        print("No patch needed: Image.ANTIALIAS not found")
else:
    print(f"Error: {file_path} does not exist")
    # Fallback: Monkey-patch
    Image.ANTIALIAS = Image.Resampling.LANCZOS
    print("Applied monkey-patch for Image.ANTIALIAS")

# Reload the patched module
import util.preprocess_itw_im
importlib.reload(util.preprocess_itw_im)

# Step 4: Download pretrained models
!python download_models.py

# Step 5: Set testing options
opt = TestOptions().parse(save=False)
opt.display_id = 0
opt.nThreads = 1
opt.batchSize = 1
opt.serial_batches = True
opt.no_flip = True
opt.in_the_wild = True
opt.traverse = True
opt.interp_step = 0.05
opt.name = 'males_model'  # Use 'females_model' for female images

# Step 6: Create data loader and model
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)
model = create_model(opt)
model.eval()

# Step 7: Upload image
uploaded = files.upload()
for filename in uploaded.keys():
    img_path = filename
    print("Uploaded:", img_path)

# Step 8: Inference
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

# Step 9: Save result as video
os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

# Step 10: Display result
mp4 = open(out_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=512 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")

# Step 11: Download result
files.download(out_path)

In [ ]:
!ls /content/Lifespan_Age_Transformation_Synthesis/
!ls /content/Lifespan_Age_Transformation_Synthesis/options/

In [ ]:
import os
print(os.getcwd())  # Should print /content/Lifespan_Age_Transformation_Synthesis

In [ ]:
%cd /content/Lifespan_Age_Transformation_Synthesis

In [ ]:
import sys
sys.path.append('/content/Lifespan_Age_Transformation_Synthesis')

In [ ]:
!pip install -r requirements.txt

In [ ]:
!cat /content/Lifespan_Age_Transformation_Synthesis/options/test_options.py

In [ ]:
import os
import sys
import importlib
from PIL import Image
from collections import OrderedDict

# Step 1: Clone the GitHub repository
!rm -rf /content/Lifespan_Age_Transformation_Synthesis  # Remove existing directory to avoid conflicts
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd /content/Lifespan_Age_Transformation_Synthesis

# Verify directory structure
!ls /content/Lifespan_Age_Transformation_Synthesis
!ls /content/Lifespan_Age_Transformation_Synthesis/options

# Step 2: Add repository to Python path
sys.path.append('/content/Lifespan_Age_Transformation_Synthesis')

# Step 3: Install required packages
!pip install -r requirements.txt

# Step 4: Patch deprecated Image.ANTIALIAS to Image.Resampling.LANCZOS
file_path = "util/preprocess_itw_im.py"
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    if "Image.ANTIALIAS" in content:
        content = content.replace("Image.ANTIALIAS", "Image.Resampling.LANCZOS")
        with open(file_path, 'w') as file:
            file.write(content)
        print("Successfully patched util/preprocess_itw_im.py")
    else:
        print("No patch needed: Image.ANTIALIAS not found")
else:
    print(f"Error: {file_path} does not exist")
    # Fallback: Monkey-patch
    Image.ANTIALIAS = Image.Resampling.LANCZOS
    print("Applied monkey-patch for Image.ANTIALIAS")

# Reload the patched module
import util.preprocess_itw_im
importlib.reload(util.preprocess_itw_im)

# Step 5: Import remaining modules
try:
    from options.test_options import TestOptions
    from data.data_loader import CreateDataLoader
    from models.models import create_model
    import util.util as util
    from util.visualizer import Visualizer
    from google.colab import files
    from IPython.display import HTML
    from base64 import b64encode
except ModuleNotFoundError as e:
    print(f"Module import failed: {e}")
    print("Ensure the repository is cloned correctly and the 'options' directory exists.")
    raise

# Step 6: Set testing options
opt = TestOptions().parse(save=False)
opt.display_id = 0
opt.nThreads = 1
opt.batchSize = 1
opt.serial_batches = True
opt.no_flip = True
opt.in_the_wild = True
opt.traverse = True
opt.interp_step = 0.05
opt.name = 'males_model'  # Use 'females_model' for female images

# Step 7: Create data loader and model
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)
model = create_model(opt)
model.eval()

# Step 8: Upload image
uploaded = files.upload()
for filename in uploaded.keys():
    img_path = filename
    print("Uploaded:", img_path)

# Step 9: Inference
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

# Step 10: Save result as video
os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

# Step 11: Display result
mp4 = open(out_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=512 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")

# Step 12: Download result
files.download(out_path)

In [ ]:
%cd /content
!pwd  # Should output /content

In [ ]:
!rm -rf /content/Lifespan_Age_Transformation_Synthesis
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd /content/Lifespan_Age_Transformation_Synthesis

In [ ]:
!ls /content/Lifespan_Age_Transformation_Synthesis
!ls /content/Lifespan_Age_Transformation_Synthesis/options
!ls /content/Lifespan_Age_Transformation_Synthesis/util

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install torch==1.9.0 torchvision==0.10.0 pillow==9.5.0

In [ ]:
!python --version
!cat /etc/os-release  # Check Linux distribution
!nvcc --version  # Check CUDA version (if using GPU)

In [ ]:
import os
import sys
import importlib
from PIL import Image
from collections import OrderedDict

# Step 1: Reset and set working directory
%cd /content
!rm -rf /content/Lifespan_Age_Transformation_Synthesis
!pwd  # Verify working directory is /content

# Step 2: Clone the GitHub repository
try:
    !git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
    print("Repository cloned successfully")
except Exception as e:
    print(f"Failed to clone repository: {e}")
    raise

# Step 3: Change to repository directory
%cd /content/Lifespan_Age_Transformation_Synthesis
!pwd  # Should output /content/Lifespan_Age_Transformation_Synthesis

# Step 4: Verify directory structure
!ls /content/Lifespan_Age_Transformation_Synthesis
!ls /content/Lifespan_Age_Transformation_Synthesis/options
!ls /content/Lifespan_Age_Transformation_Synthesis/util

# Step 5: Add repository to Python path
sys.path.append('/content/Lifespan_Age_Transformation_Synthesis')

# Step 6: Install required packages
try:
    !pip install -r requirements.txt
    print("Dependencies installed successfully")
except Exception as e:
    print(f"Failed to install dependencies from requirements.txt: {e}")
    # Install compatible versions for Python 3.11 and CUDA 12.5
    !pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 pillow==9.5.0 -f https://download.pytorch.org/whl/torch_stable.html
    print("Installed fallback dependencies: torch==2.3.0+cu121, torchvision==0.18.0+cu121, pillow==9.5.0")

# Step 7: Patch deprecated Image.ANTIALIAS to Image.Resampling.LANCZOS
file_path = "util/preprocess_itw_im.py"
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    if "Image.ANTIALIAS" in content:
        content = content.replace("Image.ANTIALIAS", "Image.Resampling.LANCZOS")
        with open(file_path, 'w') as file:
            file.write(content)
        print("Successfully patched util/preprocess_itw_im.py")
    else:
        print("No patch needed: Image.ANTIALIAS not found")
else:
    print(f"Error: {file_path} does not exist")
    # Fallback: Monkey-patch
    Image.ANTIALIAS = Image.Resampling.LANCZOS
    print("Applied monkey-patch for Image.ANTIALIAS")

# Reload the patched module
try:
    import util.preprocess_itw_im
    importlib.reload(util.preprocess_itw_im)
except ModuleNotFoundError as e:
    print(f"Failed to import util.preprocess_itw_im: {e}")
    raise

# Step 8: Import remaining modules
try:
    from options.test_options import TestOptions
    from data.data_loader import CreateDataLoader
    from models.models import create_model
    import util.util as util
    from util.visualizer import Visualizer
    from google.colab import files
    from IPython.display import HTML
    from base64 import b64encode
except ModuleNotFoundError as e:
    print(f"Module import failed: {e}")
    print("Ensure the repository is cloned correctly and the 'options' and 'util' directories exist.")
    raise

# Step 9: Download pretrained models
try:
    !python download_models.py
    print("Pretrained models downloaded successfully")
except Exception as e:
    print(f"Failed to download models: {e}")
    raise

# Step 10: Set testing options
opt = TestOptions().parse(save=False)
opt.display_id = 0
opt.nThreads = 1
opt.batchSize = 1
opt.serial_batches = True
opt.no_flip = True
opt.in_the_wild = True
opt.traverse = True
opt.interp_step = 0.05
opt.name = 'males_model'  # Use 'females_model' for female images

# Step 11: Create data loader and model
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)
model = create_model(opt)
model.eval()

# Step 12: Upload image
uploaded = files.upload()
for filename in uploaded.keys():
    img_path = filename
    print("Uploaded:", img_path)

# Step 13: Inference
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

# Step 14: Save result as video
os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

# Step 15: Display result
mp4 = open(out_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=512 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")

# Step 16: Download result
files.download(out_path)

In [ ]:
import os
import sys
import importlib
from PIL import Image
from collections import OrderedDict
from IPython.display import HTML
from base64 import b64encode

# Step 1: Reset and set working directory
%cd /content
!rm -rf /content/Lifespan_Age_Transformation_Synthesis
!pwd  # Verify working directory is /content

# Step 2: Clone the GitHub repository
try:
    !git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
    print("Repository cloned successfully")
except Exception as e:
    print(f"Failed to clone repository: {e}")
    raise

# Step 3: Change to repository directory
%cd /content/Lifespan_Age_Transformation_Synthesis
!pwd  # Should output /content/Lifespan_Age_Transformation_Synthesis

# Step 4: Verify directory structure
!ls /content/Lifespan_Age_Transformation_Synthesis
!ls /content/Lifespan_Age_Transformation_Synthesis/options
!ls /content/Lifespan_Age_Transformation_Synthesis/util

# Step 5: Add repository to Python path
sys.path.append('/content/Lifespan_Age_Transformation_Synthesis')

# Step 6: Install required packages
try:
    !pip install -r requirements.txt
    print("Dependencies installed successfully")
except Exception as e:
    print(f"Failed to install dependencies from requirements.txt: {e}")
    # Install compatible versions for Python 3.11 and CUDA 12.5
    !pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 pillow==9.5.0 -f https://download.pytorch.org/whl/torch_stable.html
    print("Installed fallback dependencies: torch==2.3.0+cu121, torchvision==0.18.0+cu121, pillow==9.5.0")

# Step 7: Patch deprecated Image.ANTIALIAS to Image.Resampling.LANCZOS
file_path = "util/preprocess_itw_im.py"
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    if "Image.ANTIALIAS" in content:
        content = content.replace("Image.ANTIALIAS", "Image.Resampling.LANCZOS")
        with open(file_path, 'w') as file:
            file.write(content)
        print("Successfully patched util/preprocess_itw_im.py")
    else:
        print("No patch needed: Image.ANTIALIAS not found")
else:
    print(f"Error: {file_path} does not exist")
    # Fallback: Monkey-patch
    Image.ANTIALIAS = Image.Resampling.LANCZOS
    print("Applied monkey-patch for Image.ANTIALIAS")

# Reload the patched module
try:
    import util.preprocess_itw_im
    importlib.reload(util.preprocess_itw_im)
except ModuleNotFoundError as e:
    print(f"Failed to import util.preprocess_itw_im: {e}")
    raise

# Step 8: Import remaining modules
try:
    from options.test_options import TestOptions
    from data.data_loader import CreateDataLoader
    from models.models import create_model
    import util.util as util
    from util.visualizer import Visualizer
    from google.colab import files
except ModuleNotFoundError as e:
    print(f"Module import failed: {e}")
    print("Ensure the repository is cloned correctly and the 'options' and 'util' directories exist.")
    raise

# Step 9: Download pretrained models
try:
    !python download_models.py
    print("Pretrained models downloaded successfully")
except Exception as e:
    print(f"Failed to download models: {e}")
    raise

# Step 10: Set testing options
opt = TestOptions().parse(save=False)
opt.display_id = 0
opt.nThreads = 1
opt.batchSize = 1
opt.serial_batches = True
opt.no_flip = True
opt.in_the_wild = True
opt.traverse = True
opt.interp_step = 0.05
opt.name = 'males_model'  # Use 'females_model' for female images

# Step 11: Create data loader and model
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)
model = create_model(opt)
model.eval()

# Step 12: Upload image
uploaded = files.upload()
for filename in uploaded.keys():
    img_path = filename
    print("Uploaded:", img_path)

# Step 13: Inference
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

# Step 14: Save result as video
os.makedirs('results', exist_ok=True)
outwor_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

# Step 15: Display result in Colab
mp4 = open(out_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=512 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")

In [ ]:
import os
import sys
import importlib
from PIL import Image
from collections import OrderedDict
from IPython.display import HTML, Video, Image as IPImage
from base64 import b64encode
import cv2  # For video inspection
import numpy as np

# Step 1: Reset and set working directory
%cd /content
!rm -rf /content/Lifespan_Age_Transformation_Synthesis
!pwd  # Verify working directory is /content

# Step 2: Clone the GitHub repository
try:
    !git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
    print("Repository cloned successfully")
except Exception as e:
    print(f"Failed to clone repository: {e}")
    raise

# Step 3: Change to repository directory
%cd /content/Lifespan_Age_Transformation_Synthesis
!pwd  # Should output /content/Lifespan_Age_Transformation_Synthesis

# Step 4: Verify directory structure
!ls /content/Lifespan_Age_Transformation_Synthesis
!ls /content/Lifespan_Age_Transformation_Synthesis/options
!ls /content/Lifespan_Age_Transformation_Synthesis/util

# Step 5: Add repository to Python path
sys.path.append('/content/Lifespan_Age_Transformation_Synthesis')

# Step 6: Install required packages
try:
    !pip install -r requirements.txt
    !pip install opencv-python  # Ensure OpenCV is installed
    print("Dependencies installed successfully")
except Exception as e:
    print(f"Failed to install dependencies from requirements.txt: {e}")
    # Install compatible versions for Python 3.11 and CUDA 12.5
    !pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 pillow==9.5.0 opencv-python -f https://download.pytorch.org/whl/torch_stable.html
    print("Installed fallback dependencies: torch==2.3.0+cu121, torchvision==0.18.0+cu121, pillow==9.5.0, opencv-python")

# Verify OpenCV installation
try:
    print(f"OpenCV version: {cv2.__version__}")
except ImportError:
    print("OpenCV not installed correctly")
    raise

# Step 7: Patch deprecated Image.ANTIALIAS to Image.Resampling.LANCZOS
file_path = "util/preprocess_itw_im.py"
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    if "Image.ANTIALIAS" in content:
        content = content.replace("Image.ANTIALIAS", "Image.Resampling.LANCZOS")
        with open(file_path, 'w') as file:
            file.write(content)
        print("Successfully patched util/preprocess_itw_im.py")
    else:
        print("No patch needed: Image.ANTIALIAS not found")
else:
    print(f"Error: {file_path} does not exist")
    # Fallback: Monkey-patch
    Image.ANTIALIAS = Image.Resampling.LANCZOS
    print("Applied monkey-patch for Image.ANTIALIAS")

# Reload the patched module
try:
    import util.preprocess_itw_im
    importlib.reload(util.preprocess_itw_im)
except ModuleNotFoundError as e:
    print(f"Failed to import util.preprocess_itw_im: {e}")
    raise

# Step 8: Import remaining modules
try:
    from options.test_options import TestOptions
    from data.data_loader import CreateDataLoader
    from models.models import create_model
    import util.util as util
    from util.visualizer import Visualizer
    from google.colab import files
except ModuleNotFoundError as e:
    print(f"Module import failed: {e}")
    print("Ensure the repository is cloned correctly and the 'options' and 'util' directories exist.")
    raise

# Step 9: Download pretrained models
try:
    !python download_models.py
    print("Pretrained models downloaded successfully")
except Exception as e:
    print(f"Failed to download models: {e}")
    raise

# Step 10: Set testing options
opt = TestOptions().parse(save=False)
opt.display_id = 0
opt.nThreads = 1
opt.batchSize = 1
opt.serial_batches = True
opt.no_flip = True
opt.in_the_wild = True
opt.traverse = True
opt.interp_step = 0.05
opt.name = 'males_model'  # Use 'females_model' for female images

# Step 11: Create data loader and model
try:
    data_loader = CreateDataLoader(opt)
    dataset = data_loader.load_data()
    visualizer = Visualizer(opt)
    model = create_model(opt)
    model.eval()
except Exception as e:
    print(f"Failed to initialize data loader or model: {e}")
    raise

# Step 12: Upload image
try:
    uploaded = files.upload()
    for filename in uploaded.keys():
        img_path = filename
        print("Uploaded:", img_path)
except Exception as e:
    print(f"Failed to upload image: {e}")
    raise

# Step 13: Inference
try:
    data = dataset.dataset.get_item_from_path(img_path)
    visuals = model.inference(data)
    print("Inference completed. Visuals keys:", list(visuals.keys()))
except Exception as e:
    print(f"Inference failed: {e}")
    raise

# Step 14: Save result as video
try:
    os.makedirs('results', exist_ok=True)
    out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
    visualizer.make_video(visuals, out_path)
    print(f"Video saved to: {out_path}")
except Exception as e:
    print(f"Failed to generate video: {e}")
    raise

# Step 15: Verify video file
if os.path.exists(out_path):
    file_size = os.path.getsize(out_path)
    print(f"Video file size: {file_size / 1024 / 1024:.2f} MB")
    if file_size < 1024:  # Check if file is too small (likely empty or corrupted)
        print("Warning: Video file is very small and may be corrupted.")
else:
    print(f"Error: Video file {out_path} does not exist.")
    raise FileNotFoundError(f"Video file {out_path} not found")

# Step 16: Check video validity with OpenCV
try:
    cap = cv2.VideoCapture(out_path)
    if cap.isOpened():
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print(f"Video details: {frame_count} frames, {fps:.2f} FPS, {width}x{height} resolution")
        cap.release()
    else:
        print("Error: Video file could not be opened with OpenCV.")
except Exception as e:
    print(f"Failed to read video with OpenCV: {e}")

# Step 17: Display result in Colab (Method 1: IPython.display.Video)
try:
    print("Displaying video using IPython.display.Video method...")
    display(Video(out_path, embed=True, width=512))
except Exception as e:
    print(f"Failed to display video with IPython.display.Video method: {e}")

# Step 18: Display result in Colab (Method 2: HTML)
try:
    with open(out_path, 'rb') as mp4:
        data_url = "data:video/mp4;base64," + b64encode(mp4.read()).decode()
    print("Displaying video using HTML method...")
    display(HTML(f"""
    <video width=512 controls>
      <source src="{data_url}" type="video/mp4">
    </video>
    """))
except Exception as e:
    print(f"Failed to display video with HTML method: {e}")

# Step 19: Fallback - Display first frame if video fails
try:
    if os.path.exists(out_path):
        cap = cv2.VideoCapture(out_path)
        ret, frame = cap.read()
        if ret:
            # Convert BGR to RGB for display
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Save frame temporarily for display
            temp_frame_path = 'results/temp_frame.jpg'
            cv2.imwrite(temp_frame_path, frame_rgb)
            print("Displaying first frame of video as fallback...")
            display(IPImage(temp_frame_path))
        else:
            print("Error: Could not read first frame of video.")
        cap.release()
    else:
        print("Error: Video file not found for frame extraction.")
except Exception as e:
    print(f"Failed to display first frame: {e}")
    print("Please check if the video file is valid. You can try downloading it manually from the Colab file explorer.")